In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

In [11]:
df = pd.read_csv('LSA.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.396314,-0.036150,0.064518,-0.073159,0.096052,-0.050067,-0.090869,0.037282,0.100209,0.046334,...,-0.024380,-0.029560,0.071205,0.114251,-0.060556,0.027602,-0.055176,-0.129919,-0.060081,0
1,0.401282,-0.186660,-0.115922,-0.267420,-0.016475,-0.064113,-0.072102,-0.035793,0.172962,-0.108011,...,0.038402,-0.041010,0.069455,-0.012704,-0.055811,0.060482,-0.017539,-0.082948,0.005765,0
2,0.421012,-0.051170,0.031429,-0.184141,-0.075076,-0.184510,-0.078020,0.011355,-0.018534,0.067499,...,-0.021454,0.005910,0.021597,-0.084544,0.045725,0.020761,0.045298,-0.053283,0.026643,0
3,0.419083,0.137935,0.029789,0.027398,0.053296,0.017428,-0.058439,-0.094601,0.061460,-0.050057,...,0.017194,-0.003981,0.106977,-0.081015,0.083304,-0.058552,0.098363,-0.012170,0.001477,0
4,0.411648,0.156481,0.054695,-0.188321,0.130529,-0.111796,0.059512,-0.038671,0.045371,-0.163627,...,-0.076978,0.039268,-0.015464,0.008226,-0.036455,-0.023845,0.070630,-0.013401,-0.080703,0


In [12]:
y = df['100']
x = df.drop('100', axis=1) 
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)

In [13]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.824615  0.662197   
1  XGBClassifier(base_score=None, booster=None, c...  0.780769  0.561731   
2      LGBMClassifier(max_depth=10, random_state=50)  0.777692  0.555864   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.783846  0.568113   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.660000  0.320038   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.649231   0.904215  0.726154  0.805461     0.923077     0.726154  
1  0.561538   0.788310  0.767692  0.777864     0.793846     0.767692  
2  0.555385   0.789727  0.756923  0.772977     0.798462     0.756923  
3  0.567692   0.773333  0.803077  0.787925     0.764615     0.803077  
4  0.320000   0.662500  0.652308  0.657364     0.667692     0.652308  
